# Data Preparation

Let us read in the different datasets.

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
base_path_train = "/home/julian/Desktop/data_science/ss22/recsys/project/dressipi_recsys2022"


items_df = pd.read_csv(os.path.join(base_path_train, "item_features.csv"))
purchase_df = pd.read_csv(os.path.join(base_path_train, "train_purchases.csv"))
session_df = pd.read_csv(os.path.join(base_path_train, "train_sessions.csv"))

In [3]:
items_df

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75
...,...,...,...
471746,28143,68,351
471747,28143,55,390
471748,28143,11,109
471749,28143,73,91


In [4]:
items_df.item_id.nunique()

23691

In [5]:
purchase_df

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114
...,...,...,...
999995,4439986,2915,2021-05-13 11:56:37.464
999996,4439990,8786,2020-08-22 14:28:22.382
999997,4439994,21630,2020-11-27 20:10:28.961
999998,4439999,16962,2020-11-27 11:01:41.356


In [6]:
session_df

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211
...,...,...,...
4743815,4440001,20409,2020-10-30 23:37:20.658
4743816,4440001,14155,2020-10-30 23:31:56.607
4743817,4440001,14303,2020-10-30 23:36:17.934
4743818,4440001,27852,2020-10-30 23:39:55.186


Now we combine the views inside a session and the purchases of this session in one dataframe, with the column `was_bought` indicating whether the item was only viewed or bought.

In [7]:
purchase_df_processed = purchase_df.copy()
purchase_df_processed["was_bought"] = True

session_df_processed = session_df.copy()
session_df_processed["was_bought"] = False
df_processed = pd.concat([purchase_df_processed, session_df_processed]).sort_values(["session_id", "date"])
df_processed

,session_id,item_id,date,was_bought
1,3,9655,2020-12-18 21:19:48.093,False
0,3,9655,2020-12-18 21:25:00.373,False
0,3,15085,2020-12-18 21:26:47.986,True
2,13,15654,2020-03-13 19:35:27.136,False
1,13,18626,2020-03-13 19:36:15.507,True
...,...,...,...,...
4743804,4440001,19539,2020-10-30 23:37:09.46,False
4743815,4440001,20409,2020-10-30 23:37:20.658,False
4743818,4440001,27852,2020-10-30 23:39:55.186,False
4743806,4440001,20449,2020-10-30 23:40:28.149,False


Now we denormalize the item features table, to have a more handy representation of the item features

In [8]:
items_processed_df = items_df.pivot_table(values='feature_value_id', index='item_id', columns='feature_category_id').reset_index()
items_processed_df.index.names = ['index']
items_processed_df.columns = ["item_id"] + [f"item_feature_{x}" for x in list(range(73))]
items_processed_df

,item_id,item_feature_0,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,item_feature_7,item_feature_8,...,item_feature_63,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72
index,,,,,,,,,,,,,,,,,,,,,
0,2,NaN,NaN,NaN,NaN,NaN,NaN,394.0,NaN,NaN,...,NaN,NaN,NaN,NaN,351.0,885.0,NaN,NaN,75.0,NaN
1,3,NaN,NaN,889.0,618.0,605.0,NaN,452.0,NaN,NaN,...,NaN,521.0,NaN,NaN,14.0,592.0,NaN,NaN,75.0,544.0
2,4,NaN,NaN,793.0,618.0,605.0,NaN,837.0,NaN,NaN,...,NaN,521.0,NaN,NaN,373.0,538.0,NaN,NaN,75.0,544.0
3,7,NaN,NaN,NaN,NaN,NaN,NaN,536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,739.0,592.0,NaN,NaN,75.0,NaN
4,8,NaN,NaN,793.0,618.0,605.0,NaN,798.0,NaN,NaN,...,NaN,521.0,NaN,NaN,351.0,592.0,NaN,NaN,75.0,544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,NaN,NaN,793.0,618.0,605.0,NaN,798.0,NaN,NaN,...,NaN,521.0,NaN,NaN,106.0,805.0,NaN,NaN,75.0,544.0
23687,28140,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.0,NaN,NaN,349.0,351.0,NaN,NaN,226.0,NaN,544.0
23688,28141,461.0,NaN,889.0,719.0,605.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,379.0,499.0,NaN,NaN,75.0,544.0


The item features can now be merged to the combined dataset with session views and purchases from above.
Also NULL values are filled by 0.

In [9]:
df_processed = df_processed.merge(items_processed_df, how="left", on="item_id")
df_processed["was_bought"] = df_processed["was_bought"].astype(float)
df_processed

,session_id,item_id,date,was_bought,item_feature_0,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,...,item_feature_63,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72
0,3,9655,2020-12-18 21:19:48.093,0.0,NaN,53.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,349.0,393.0,NaN,NaN,NaN,NaN,544.0
1,3,9655,2020-12-18 21:25:00.373,0.0,NaN,53.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,349.0,393.0,NaN,NaN,NaN,NaN,544.0
2,3,15085,2020-12-18 21:26:47.986,1.0,NaN,53.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,349.0,97.0,NaN,NaN,NaN,NaN,544.0
3,13,15654,2020-03-13 19:35:27.136,0.0,NaN,NaN,NaN,618.0,NaN,766.0,...,NaN,521.0,NaN,NaN,351.0,780.0,NaN,NaN,219.0,NaN
4,13,18626,2020-03-13 19:36:15.507,1.0,NaN,NaN,793.0,618.0,605.0,NaN,...,NaN,NaN,NaN,NaN,739.0,805.0,NaN,NaN,75.0,544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5743815,4440001,19539,2020-10-30 23:37:09.46,0.0,NaN,NaN,NaN,618.0,NaN,NaN,...,NaN,NaN,NaN,NaN,351.0,885.0,NaN,NaN,75.0,544.0
5743816,4440001,20409,2020-10-30 23:37:20.658,0.0,NaN,NaN,NaN,618.0,NaN,NaN,...,NaN,NaN,NaN,NaN,351.0,885.0,NaN,NaN,75.0,544.0
5743817,4440001,27852,2020-10-30 23:39:55.186,0.0,NaN,NaN,NaN,618.0,NaN,778.0,...,NaN,550.0,NaN,NaN,351.0,362.0,NaN,NaN,75.0,544.0
5743818,4440001,20449,2020-10-30 23:40:28.149,0.0,NaN,NaN,NaN,618.0,NaN,778.0,...,NaN,550.0,NaN,NaN,351.0,362.0,NaN,NaN,75.0,544.0


In [10]:
items_processed_df = items_processed_df.fillna(0)
items_processed_df

,item_id,item_feature_0,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,item_feature_7,item_feature_8,...,item_feature_63,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72
index,,,,,,,,,,,,,,,,,,,,,
0,2,0.0,0.0,0.0,0.0,0.0,0.0,394.0,0.0,0.0,...,0.0,0.0,0.0,0.0,351.0,885.0,0.0,0.0,75.0,0.0
1,3,0.0,0.0,889.0,618.0,605.0,0.0,452.0,0.0,0.0,...,0.0,521.0,0.0,0.0,14.0,592.0,0.0,0.0,75.0,544.0
2,4,0.0,0.0,793.0,618.0,605.0,0.0,837.0,0.0,0.0,...,0.0,521.0,0.0,0.0,373.0,538.0,0.0,0.0,75.0,544.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,536.0,0.0,0.0,...,0.0,0.0,0.0,0.0,739.0,592.0,0.0,0.0,75.0,0.0
4,8,0.0,0.0,793.0,618.0,605.0,0.0,798.0,0.0,0.0,...,0.0,521.0,0.0,0.0,351.0,592.0,0.0,0.0,75.0,544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,0.0,0.0,793.0,618.0,605.0,0.0,798.0,0.0,0.0,...,0.0,521.0,0.0,0.0,106.0,805.0,0.0,0.0,75.0,544.0
23687,28140,0.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,80.0,0.0,0.0,349.0,351.0,0.0,0.0,226.0,0.0,544.0
23688,28141,461.0,0.0,889.0,719.0,605.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,379.0,499.0,0.0,0.0,75.0,544.0


In [11]:
item_id2index = dict(zip(items_processed_df.item_id, items_processed_df.index))

In [12]:
all_items = list(items_processed_df["item_id"])

In [13]:
items_processed_array = np.array(items_processed_df.drop("item_id",axis=1))
items_processed_array[item_id2index[2]]

array([  0.,   0.,   0.,   0.,   0.,   0., 394.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,  38.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0., 123.,   0.,   0.,   0., 802.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0., 123.,   0.,   0.,  76.,   0.,   0.,   6.,   0.,   0.,
       365.,   0.,   0.,   0.,   0., 462., 801.,   0.,   0.,   0.,   0.,
         0., 351., 885.,   0.,   0.,  75.,   0.])

Next we read in the candidate items.

In [14]:
candidate_items = list(pd.read_csv("candidate_items.csv")["item_id"])
candidate_items[:10]

[4, 8, 9, 19, 20, 26, 33, 40, 51, 54]

# Word2vec

First, let us prepare the training data for the LSTM. For this, we select a subset of the training dataset, so that RAM is not exceeded.

In [15]:
session_ids_cand = df_processed[(df_processed.was_bought==1) & (df_processed.item_id.isin(candidate_items))].session_id

In [16]:
#df_w2v = df_processed[df_processed.session_id < 10000].sort_values(["session_id", "date"])[["session_id", "item_id", "was_bought"]]
df_w2v = df_processed[(df_processed.session_id.isin(session_ids_cand)) & (df_processed.session_id < 50000)].sort_values(["session_id", "date"])[["session_id", "item_id", "was_bought"]]
df_w2v

,session_id,item_id,was_bought
3,13,15654,0.0
4,13,18626,1.0
42,31,25972,0.0
43,31,16289,0.0
44,31,2069,0.0
...,...,...,...
64212,49975,2072,0.0
64213,49975,2072,0.0
64214,49975,23565,1.0
64224,49993,19048,0.0


In the next step, we just map the item IDs to a subsequent list of numbers (without wholes)

In [17]:
item_list = list(df_w2v.item_id.sort_values().unique())
item_dict = {v: k for k, v in dict(zip(range(len(item_list)), item_list)).items()}
item_dict[4]

1

In [18]:
item_dict_rev = {v: k for k, v in item_dict.items()}

In the next step, we extract the viewed items (corresponding to the previous words) and purchased items (corresponding to the next words).

In [19]:
def get_values(x):
    x = x["item_id"]
    return [item_dict[x] for x in list(x.values.ravel())]

all_words = df_w2v.groupby('session_id').apply(get_values).to_list()

def get_values(x):
    x = x["item_id"]
    return [item_dict[x] for x in list(x.values.ravel())]

prev_words = df_w2v[(df_w2v.was_bought==0)].groupby('session_id').apply(get_values).to_list()

def get_values(x):
    x = x["item_id"]
    return item_dict[x.values.ravel()[0]]

next_words = df_w2v[(df_w2v.was_bought==1)].groupby('session_id').apply(get_values).to_list()

In [20]:
prev_words[:5]

[[3943],
 [6554, 4100, 519, 519, 6688, 63, 6983, 1105],
 [5219, 5698, 2622, 4410],
 [1614, 5014, 540, 3968, 5381, 4338],
 [5002, 2634, 4018, 5002, 2634, 4187, 5002, 4187]]

In [21]:
next_words[:5]

[4714, 2148, 2993, 5360, 6556]

In [22]:
unique_words = [item_dict[x] for x in df_w2v["item_id"].unique()]
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))
len(unique_words)

7130

Sessions do usually have different lengths. Thus we take the longest session as standard session length and pad the shorter sessions with zeros in the beginning, as follows:

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
max_seq_length = max([len(x) for x in prev_words])
input_seqs = np.array(pad_sequences(prev_words, maxlen=max_seq_length, padding='pre'))

print(max_seq_length)
print(input_seqs[:5])

90
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0 3943]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0 6554 4100
   519  519 6688   63 6983 1105]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 

To reprensent each item by some feature vector, we use Word2Vec embedding. From this we build an embedding matrix.

In [26]:
from gensim.models.word2vec import Word2Vec

In [27]:
w2v = Word2Vec(all_words, min_count=1)

In [28]:
#list the vocabulary words
words = list(w2v.wv.index_to_key)

print(words[:5])

[6799, 2078, 5052, 2054, 5274]


In [29]:
my_dict = dict({})
for idx, key in enumerate(w2v.wv.index_to_key):
    my_dict[key] = w2v.wv[key]
    # Or my_dict[key] = model.wv.get_vector(key)
    # Or my_dict[key] = model.wv.word_vec(key, use_norm=False)

In [30]:
embeddings_matrix = np.zeros((len(words), 100))

for i, word in enumerate(words):
    embedding_vector = my_dict[word]
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [31]:
embeddings_matrix

array([[-0.00841014,  0.01240021,  0.01478981, ..., -0.02749958,
         0.00778089,  0.00730486],
       [-0.01381387,  0.01190782,  0.00910956, ..., -0.01235555,
        -0.00937143,  0.01076616],
       [-0.00423272,  0.01369903, -0.00569892, ..., -0.00649391,
         0.01068221, -0.00357176],
       ...,
       [ 0.0058262 ,  0.00362978, -0.00626381, ..., -0.00658904,
         0.00909166, -0.005718  ],
       [-0.00585137,  0.0053937 , -0.00424384, ..., -0.00776739,
         0.00027181,  0.00699548],
       [-0.00617116,  0.00636265,  0.00508853, ...,  0.00673201,
         0.00094805,  0.00984803]])

In [32]:
embeddings_matrix.shape

(7130, 100)

Let us define our LSTM model.

In [33]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD

In [36]:
model = tf.keras.Sequential(
    [tf.keras.layers.Embedding(input_dim = len(words), output_dim=100, weights=[embeddings_matrix], input_length=max_seq_length, trainable=False),
     tf.keras.layers.LSTM(256),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(len(words) , activation='softmax')
    ])

opt = SGD(lr=10**(-6))
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Also, we convert our output (i.e. the purchased items) to a categorical object.

In [41]:
next_words = np.array(next_words)
outputs = tf.keras.utils.to_categorical(next_words, num_classes=len(words))
outputs.shape

(5042, 7130)

Now, we train our model.

In [42]:
tf.config.run_functions_eagerly(True)

In [43]:
history = model.fit(input_seqs, outputs, epochs=10, validation_split=0.2, verbose=1, batch_size=256)

/home/julian/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


Epoch 1/10
16/16 [==============================] - 8s 513ms/step - loss: 8.8721 - accuracy: 0.0000e+00 - val_loss: 8.8721 - val_accuracy: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 11s 657ms/step - loss: 8.8721 - accuracy: 0.0000e+00 - val_loss: 8.8721 - val_accuracy: 0.0000e+00
Epoch 3/10
16/16 [==============================] - 11s 674ms/step - loss: 8.8721 - accuracy: 0.0000e+00 - val_loss: 8.8721 - val_accuracy: 0.0000e+00
Epoch 4/10
16/16 [==============================] - 11s 684ms/step - loss: 8.8721 - accuracy: 0.0000e+00 - val_loss: 8.8721 - val_accuracy: 0.0000e+00
Epoch 5/10
16/16 [==============================] - 10s 621ms/step - loss: 8.8721 - accuracy: 0.0000e+00 - val_loss: 8.8721 - val_accuracy: 0.0000e+00
Epoch 6/10
16/16 [==============================] - 10s 618ms/step - loss: 8.8721 - accuracy: 2.4795e-04 - val_loss: 8.8721 - val_accuracy: 0.0000e+00
Epoch 7/10
16/16 [==============================] - 10s 630ms/step - loss: 8.8721 - accuracy: 2

# Prediction on test data

Let us read in the test data set.

In [64]:
import pandas as pd
import os
import numpy as np

base_path_test = "/home/julian/Desktop/data_science/ss22/recsys/project/dressipi_recsys2022"


test_df = pd.read_csv(os.path.join(base_path_test, "test_final_sessions.csv"))

test_df.session_id.max()

4439966

In order to not exceed the RAM, we split the test dataframe into sub-dataframes, which will be predicted batch by batch.

In [65]:
session_id_limits = list(range(0, test_df.session_id.max() + 10000000, 10000000))

In [66]:
test_dfs = []
for i in range(len(session_id_limits)-1):
    session_id_limit1 = session_id_limits[i]
    session_id_limit2 = session_id_limits[i+1]
    test_df_small = test_df[(test_df.session_id >= session_id_limit1) & (test_df.session_id < session_id_limit2)]
    test_dfs.append(test_df_small)

In [67]:
len(test_dfs)

226138

In [68]:
def get_values(x):
    x = x["item_id"]
    return [item_dict[x] for x in list(x.values.ravel()) if x in item_dict.keys()]

Now we predict using our LSTM model. For this we need to prepare the data in the same fashion as above.
From the result, we extract the 100 items with the largest scores.

In [69]:
candidate_rank_dfs = []
j = 0
for test_df_small in test_dfs:
    print(j)
    j=1
    
    # data preparation
    prev_words_test = test_df_small.groupby('session_id').apply(get_values).to_list()
    input_seqs_test = np.array(pad_sequences(prev_words_test, maxlen=max_seq_length, padding='pre'))
    
    # lstm prediction
    preds = model.predict(input_seqs_test)

    # sorting the predictions by highest score
    arr = preds.argsort().astype("float32")
    # retrieving only the candidate items
    cond = np.isin(arr.astype(int), candidate_items_keys)
    arr[~cond] = np.nan
    arrlist = arr.tolist()
    
    # creating the ranks
    for i in range(len(test_df_small.session_id.unique())):
        rank_dict = {}
        session_id  = test_df_small.session_id.unique()[i]

        # get the scores from the lecture
        scores = arrlist[i]
        y = [x for x in scores if x>=0 and x in item_dict.keys()] # x>=0 removes the nan values
        ranked_list = [item_dict_rev[x] for x in y[-100:]] # get the top 100 items

        for num, item in enumerate(ranked_list):
            rank_dict[item] = len(ranked_list) - num # this is just mapping the item to its rank
        
        candidate_rank_df = pd.DataFrame(rank_dict.items(), columns = ["item_id", "rank"]).sort_values("rank")
        candidate_rank_df["session_id"] = session_id
        candidate_rank_dfs.append(candidate_rank_df)
    
candidate_rank_df = pd.concat(candidate_rank_dfs)
candidate_rank_df = candidate_rank_df[["session_id", "item_id", "rank"]].reset_index(drop=True)
candidate_rank_df

0


/home/julian/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


,session_id,item_id,rank
0,61,5986,1
1,61,4739,2
2,61,16922,3
3,61,19,4
4,61,4065,5
...,...,...,...
4999995,4439966,18028,96
4999996,4439966,12190,97
4999997,4439966,15000,98
4999998,4439966,17638,99


In [70]:
candidate_rank_df

,session_id,item_id,rank
0,61,5986,1
1,61,4739,2
2,61,16922,3
3,61,19,4
4,61,4065,5
...,...,...,...
4999995,4439966,18028,96
4999996,4439966,12190,97
4999997,4439966,15000,98
4999998,4439966,17638,99


Finally, we write the result to a csv file.

In [74]:
candidate_rank_df.to_csv("results_lstm.csv", index=False)

# Final checks

Number of predicted sessions.

In [71]:
len(candidate_rank_df.session_id.unique())

50000

Number of recommendations of non-candidate items. Should be 0.

In [72]:
(~candidate_rank_df.item_id.isin(candidate_items)).sum()

0

Number of sessions to predict.

In [73]:
len(test_df.session_id.unique())

50000

List of items ranked first at least once.

In [75]:
candidate_rank_df[candidate_rank_df["rank"] == 1].item_id.unique()

array([ 5986,    19, 10585,  4739,  8696, 16142,  3158,  7244, 14136,
        4065, 16922,  4276, 22011,  6322, 22651, 13299, 12190, 13966,
        4141, 10287,  1611,  3593,  1917, 23938, 11697,  7037,  8449,
       18641,    64,  4430,  3771, 18415, 26882,  1821, 26451, 19472,
       20346, 11679,  7826, 10330, 21719, 25787, 21918, 13890, 17789,
        6071, 18440,  4598, 20316, 16600, 21434, 16796,  3256, 16361,
       16145,  5191, 17284,  1756,  8220,  6743, 21383, 27822, 24058,
        4250, 25537, 17925, 14455,  3453,  7502, 16660, 20082, 14336,
       24834, 19057, 23104, 27296, 23010,  7250, 15000, 19459, 25668,
       17722,  7246,  9843, 25067, 16860, 17574, 13820, 11372, 19990,
        9394, 27216, 22512, 16416,  7444, 20510, 14097, 19024,  2652,
       10676, 16232, 24076,  2303,  3704, 18028,  1102, 17316, 14130,
       24748, 20205, 17923,  6786,  6732, 11040, 10931, 15665,  2717,
       26985, 27805, 23065, 15728, 12058,  6536, 21958, 23494, 26054,
       27405, 18942,